## 0. Set up

In [35]:
! pip install transformers datasets evaluate scikit-learn
! pip install accelerate -U

In [36]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [37]:
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import numpy as np

## 1. Load dataset
I will use the HC3 (Human ChatGPT Comparison Corpus) dataset from the 🤗 Datasets library.

In [38]:
ds = load_dataset("hello-simpleai/hc3", data_files=['all.jsonl' ])
print(f"Train dataset size: {len(ds['train'])}")

Train dataset size: 24322


There are five fields in this dataset:
  - question
  - human_answers (several)
  - chatgpt_answers (several)
  - index
  - source

*** There is only train set, therefore test set should be seperated from the train set manually.

In [39]:
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'human_answers', 'chatgpt_answers', 'index', 'source'],
        num_rows: 24322
    })
})

In [40]:
ds['train'][5]['human_answers'][0]

'Melanin ! Many of the the first known humans existed in the fertile crescent - modern day Iraq and surrounding areas , and it was just as sunny and hot as it is today . Melanin causes skin and eyes to have a darker color , and as a benefit reduced the amount of UV radiation absorbed into the skin . Eventually humans expanded into less hot and sunlit areas allowing for the survival and procreation of people who developed lighter colored eyes and skin because of the lack of need of melanin for survival .'

In [41]:
# test tokenizer

roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
text = "Hello everyone!"
vocab = {v:k for k,v in roberta_tokenizer.vocab.items()}

print([(id, vocab[id]) for id in roberta_tokenizer(text)['input_ids']])
print([(id, vocab[id]) for id in roberta_tokenizer(ds['train'][3]['human_answers'][0])['input_ids']])

[(0, '<s>'), (31414, 'Hello'), (961, 'Ġeveryone'), (328, '!'), (2, '</s>')]
[(0, '<s>'), (1185, 'You'), (6056, 'Ġca'), (295, 'Ġn'), (75, "'t"), (95, 'Ġjust'), (213, 'Ġgo'), (198, 'Ġaround'), (39257, 'Ġassass'), (15647, 'inating'), (5, 'Ġthe'), (917, 'Ġleaders'), (9, 'Ġof'), (749, 'Ġcountries'), (47, 'Ġyou'), (109, 'Ġdo'), (295, 'Ġn'), (90, 't'), (101, 'Ġlike'), (27785, 'Ġ!'), (20, 'ĠThe'), (758, 'Ġinternational'), (19973, 'Ġcondemnation'), (74, 'Ġwould'), (28, 'Ġbe'), (8513, 'Ġbrutal'), (479, 'Ġ.'), (1648, 'ĠEven'), (600, 'Ġthough'), (117, 'Ġno'), (1264, 'one'), (3829, 'Ġlikes'), (1636, 'ĠKim'), (6465, 'ĠJong'), (111, 'Ġ-'), (1890, 'ĠUn'), (2156, 'Ġ,'), (8, 'Ġand'), (961, 'Ġeveryone'), (4265, 'Ġthinks'), (369, 'ĠNorth'), (1101, 'ĠKorea'), (16, 'Ġis'), (1256, 'Ġpretty'), (43816, 'Ġshitty'), (7, 'Ġto'), (63, 'Ġits'), (2286, 'Ġcitizens'), (2156, 'Ġ,'), (114, 'Ġif'), (224, 'Ġsay'), (5, 'Ġthe'), (382, 'ĠUS'), (58, 'Ġwere'), (7, 'Ġto'), (2142, 'Ġsend'), (3525, 'Ġagents'), (81, 'Ġover'), (36,

## 2. Preprocess
The next step is to load a tokenizer to preprocess the text field. A tokenizer converts text to a sequence of tokens and creates numerical representation.

In [42]:
question_list = []
human_answer_list = []
index_list = []
source_list = []
input_ids_list = []
attention_mask_list = []
label_list = []

for sample in ds['train']:

  # some of the question does not have answer.
  if len(sample['human_answers'])!=0:
    one_chatgpt_answer = sample['human_answers'][0]

    question_list.append(sample['question'])
    human_answer_list.append([one_chatgpt_answer])
    index_list.append(sample['index'])
    source_list.append(sample['source'])
    input_ids_list.append(roberta_tokenizer(one_chatgpt_answer, truncation=True)['input_ids'])
    attention_mask_list.append(roberta_tokenizer(one_chatgpt_answer, truncation=True)['attention_mask'])
    label_list.append(1)

In [43]:
tokenized_human_answers_df = pd.DataFrame.from_dict({'question':question_list, 'answer':human_answer_list, 'index':index_list, \
                                                       'source':source_list, 'input_ids':input_ids_list, 'attention_mask':attention_mask_list, \
                                                      'label':label_list})

In [44]:
question_list = []
chatgpt_answer_list = []
index_list = []
source_list = []
input_ids_list = []
attention_mask_list = []
label_list = []

for sample in ds['train']:

  # some of the question does not have answer.
  if len(sample['chatgpt_answers'])!=0:
    one_chatgpt_answer = sample['chatgpt_answers'][0]

    question_list.append(sample['question'])
    chatgpt_answer_list.append([one_chatgpt_answer])
    index_list.append(sample['index'])
    source_list.append(sample['source'])
    input_ids_list.append(roberta_tokenizer(one_chatgpt_answer, truncation=True)['input_ids'])
    attention_mask_list.append(roberta_tokenizer(one_chatgpt_answer, truncation=True)['attention_mask'])
    label_list.append(1)

In [45]:

tokenized_chatgpt_answers_df = pd.DataFrame.from_dict({'question':question_list, 'answer':chatgpt_answer_list, 'index':index_list, \
                                                       'source':source_list, 'input_ids':input_ids_list, 'attention_mask':attention_mask_list, \
                                                      'label':label_list})

In [46]:
def add_token_padding(sample):

  if len(sample)<512:

    token_paddings = [0 for i in range(512-len(sample))]
    return sample + token_paddings

  else:
    return sample

def add_mask_padding(sample):

  if len(sample)<512:

    mask_paddings = [1 for i in range(512-len(sample))]
    return sample + mask_paddings

  else:
    return sample

In [47]:
tokenized_chatgpt_answers_df['input_ids'] = tokenized_chatgpt_answers_df['input_ids'].map(lambda x: add_token_padding(x))
tokenized_chatgpt_answers_df['attention_mask'] = tokenized_chatgpt_answers_df['attention_mask'].map(lambda x: add_mask_padding(x))

In [48]:
tokenized_human_answers_df['input_ids'] = tokenized_human_answers_df['input_ids'].map(lambda x: add_token_padding(x))
tokenized_human_answers_df['attention_mask'] = tokenized_human_answers_df['attention_mask'].map(lambda x: add_mask_padding(x))

In [49]:
combined_df = pd.concat([tokenized_human_answers_df, tokenized_chatgpt_answers_df], ignore_index=True)

In [50]:
combined_df.head()

,question,answer,index,source,input_ids,attention_mask,label
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",NaN,reddit_eli5,"[0, 34480, 89, 32, 171, 6363, 9, 22, 2700, 447...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,NaN,reddit_eli5,"[0, 29, 3967, 16, 205, 13, 45, 8180, 11, 512, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,NaN,reddit_eli5,"[0, 133, 169, 24, 1364, 16, 14, 793, 1012, 449...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,NaN,reddit_eli5,"[0, 1185, 6056, 295, 75, 95, 213, 198, 39257, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,NaN,reddit_eli5,"[0, 38576, 154, 7, 3549, 5, 15328, 66, 9, 1841...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [51]:
print(tokenized_human_answers_df.shape, tokenized_chatgpt_answers_df.shape)
print(combined_df.shape)
combined_df.head()

(24322, 7) (23867, 7)
(48189, 7)


,question,answer,index,source,input_ids,attention_mask,label
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",NaN,reddit_eli5,"[0, 34480, 89, 32, 171, 6363, 9, 22, 2700, 447...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,NaN,reddit_eli5,"[0, 29, 3967, 16, 205, 13, 45, 8180, 11, 512, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,NaN,reddit_eli5,"[0, 133, 169, 24, 1364, 16, 14, 793, 1012, 449...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,NaN,reddit_eli5,"[0, 1185, 6056, 295, 75, 95, 213, 198, 39257, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,NaN,reddit_eli5,"[0, 38576, 154, 7, 3549, 5, 15328, 66, 9, 1841...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [52]:
tokenized_dataset = Dataset.from_pandas(combined_df)

In [53]:
tokenized_dataset = tokenized_dataset.remove_columns(["question", "answer", "index", "source"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

In [54]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.95)['train'].train_test_split(test_size=0.2)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_dataset['train'], shuffle=True, batch_size=20)
eval_dataloader = DataLoader(tokenized_dataset['test'], batch_size=20)

In [ ]:
id2label = {0: "human", 1: "chatgpt"}
label2id = {"human": 0, "chatgpt": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# This automodel class gives us the model with pretrained weights + a sequence classification head
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 50
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
from tqdm.auto import tqdm
import evaluate

progress_bar = tqdm(range(num_training_steps))

train_loss_list = []
val_loss_list = []


for epoch in range(num_epochs):

    train_loss = 0
    train_acc = 0
    train_total_size = 0

    for batch in train_dataloader:

        train_total_size += len(batch['input_ids'])

        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        train_loss+=loss

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)




    val_loss = 0
    val_total_size = 0

    metric = evaluate.load("accuracy")

    for batch in eval_dataloader:

        val_total_size += len(batch['input_ids'])


        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        val_loss+=loss

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])


    print(f'\tTrain Loss: {train_loss/train_total_size} | Val Loss: {val_loss/val_total_size}')

    train_loss_list.append(train_loss/train_total_size)
    val_loss_list.append(val_loss/val_total_size)



In [ ]:
train_losses_list = []
val_losses_list = []

for i in losses_list:
  train_losses_list.append(i[0])
  val_losses_list.append(i[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses_list[1:])
plt.plot(val_losses_list[1:])

In [ ]:
training_args = TrainingArguments(
    batch_size=32,
    num_train_epochs=2
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

class CustomTrainer(Trainer):

  def _inner_training_loop(
      self, args=None):

    number_of_epochs = num_epochs

    train_loss_list = []
    val_loss_list = []

    self.optimizer = AdamW(model.parameters(), lr=5e-5)

    train_dataloader = DataLoader(tokenized_dataset['train'], shuffle=True, batch_size=args.batch_size)
    eval_dataloader = DataLoader(tokenized_dataset['test'], batch_size=args.batch_size)

    self.lr_scheduler = get_scheduler(
        name="linear", optimizer=self.optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
    )

    for epoch in range(num_epochs):

        train_loss = 0
        train_acc = 0
        train_total_size = 0

        for batch in train_dataloader:

            train_total_size += len(batch['input_ids'])

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            train_loss+=loss

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)




        val_loss = 0
        val_total_size = 0

        metric = evaluate.load("accuracy")

        for batch in eval_dataloader:

            val_total_size += len(batch['input_ids'])


            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            loss = outputs.loss
            val_loss+=loss

            # logits = outputs.logits
            # predictions = torch.argmax(logits, dim=-1)
            # metric.add_batch(predictions=predictions, references=batch["labels"])


        print(f'\tTrain Loss: {train_loss/train_total_size} | Val Loss: {val_loss/val_total_size}')

        train_loss_list.append(train_loss/train_total_size)
        val_loss_list.append(val_loss/val_total_size)

